### Load the .env file

In [2]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

### Import libraries

In [3]:
import os

### Create Openai client

In [4]:
from openai import OpenAI

In [5]:
client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY')
)

### Create an assistant

In [6]:
client.beta.assistants.list()

SyncCursorPage[Assistant](data=[Assistant(id='asst_CFlbjghfoTvF4XUu3zecUNPh', created_at=1700200026, description=None, file_ids=['file-m7OnOdzjpxUM0xpF5DluUoxl'], instructions="You are a real estate agent, Use your knowledge base to best respond to customer queries. If you don't find the properties from the knowledge reply generously. Never mention that you are getting the information from the documents.", metadata={}, model='gpt-4-1106-preview', name='Real Estate', object='assistant', tools=[ToolRetrieval(type='retrieval')])], object='list', first_id='asst_CFlbjghfoTvF4XUu3zecUNPh', last_id='asst_CFlbjghfoTvF4XUu3zecUNPh', has_more=False)

In [10]:
files = client.files.list()
print(files.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "file-m7OnOdzjpxUM0xpF5DluUoxl",
      "bytes": 29264,
      "created_at": 1700199919,
      "filename": "real-estate.pdf",
      "object": "file",
      "purpose": "assistants",
      "status": "processed",
      "status_details": null
    }
  ],
  "object": "list",
  "has_more": false
}


In [15]:
real_estate = client.beta.assistants.create(
    instructions="You are a real estate agent, Use your knowledge base to best respond to customer queries.",
    name="Real Estate",
    tools=[{"type": "retrieval"}],
    model="gpt-4-1106-preview",
    file_ids=["file-m7OnOdzjpxUM0xpF5DluUoxl"]
)

In [16]:
print(real_estate.model_dump_json(indent=2))

{
  "id": "asst_CFlbjghfoTvF4XUu3zecUNPh",
  "created_at": 1700200026,
  "description": null,
  "file_ids": [
    "file-m7OnOdzjpxUM0xpF5DluUoxl"
  ],
  "instructions": "You are a real estate agent, Use your knowledge base to best respond to customer queries.",
  "metadata": {},
  "model": "gpt-4-1106-preview",
  "name": "Real Estate",
  "object": "assistant",
  "tools": [
    {
      "type": "retrieval"
    }
  ]
}


### Create a new thread for the conversation

In [17]:
thread = client.beta.threads.create()

In [18]:
print(thread.model_dump_json(indent=2))

{
  "id": "thread_jNDiXMgwUxyAi6XNLBGkCsTd",
  "created_at": 1700200038,
  "metadata": {},
  "object": "thread"
}


### Add message to the thread

In [19]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="do you have 5bhk in paldi?",
    file_ids=["file-m7OnOdzjpxUM0xpF5DluUoxl"]
)

In [20]:
print(message.model_dump_json(indent=2))

{
  "id": "msg_vQLbegepbnR3nbgu0RTJtT4m",
  "assistant_id": null,
  "content": [
    {
      "text": {
        "annotations": [],
        "value": "do you have 5bhk in paldi?"
      },
      "type": "text"
    }
  ],
  "created_at": 1700200071,
  "file_ids": [
    "file-m7OnOdzjpxUM0xpF5DluUoxl"
  ],
  "metadata": {},
  "object": "thread.message",
  "role": "user",
  "run_id": null,
  "thread_id": "thread_jNDiXMgwUxyAi6XNLBGkCsTd"
}


### Run the query

In [21]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=real_estate.id,
  instructions="Please address the user as Raj."
)

In [22]:
print(run.model_dump_json(indent=2))

{
  "id": "run_awvPEMIl1Ixe4tNcT6qDQHUH",
  "assistant_id": "asst_CFlbjghfoTvF4XUu3zecUNPh",
  "cancelled_at": null,
  "completed_at": null,
  "created_at": 1700200090,
  "expires_at": 1700200690,
  "failed_at": null,
  "file_ids": [
    "file-m7OnOdzjpxUM0xpF5DluUoxl"
  ],
  "instructions": "Please address the user as Raj.",
  "last_error": null,
  "metadata": {},
  "model": "gpt-4-1106-preview",
  "object": "thread.run",
  "required_action": null,
  "started_at": null,
  "status": "queued",
  "thread_id": "thread_jNDiXMgwUxyAi6XNLBGkCsTd",
  "tools": [
    {
      "type": "retrieval"
    }
  ]
}


### Check run status

In [25]:
retribed_run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

In [26]:
print(retribed_run.model_dump_json(indent=2))

{
  "id": "run_awvPEMIl1Ixe4tNcT6qDQHUH",
  "assistant_id": "asst_CFlbjghfoTvF4XUu3zecUNPh",
  "cancelled_at": null,
  "completed_at": 1700200100,
  "created_at": 1700200090,
  "expires_at": null,
  "failed_at": null,
  "file_ids": [
    "file-m7OnOdzjpxUM0xpF5DluUoxl"
  ],
  "instructions": "Please address the user as Raj.",
  "last_error": null,
  "metadata": {},
  "model": "gpt-4-1106-preview",
  "object": "thread.run",
  "required_action": null,
  "started_at": 1700200091,
  "status": "completed",
  "thread_id": "thread_jNDiXMgwUxyAi6XNLBGkCsTd",
  "tools": [
    {
      "type": "retrieval"
    }
  ]
}


### Show the response

In [27]:
thread_messages = client.beta.threads.messages.list(thread.id)

In [28]:
user = ''
assistant = ''

for tm in reversed(thread_messages.data):
    content = tm.content[0].text.value
    if tm.role == 'user':
        user += content
    if tm.role == 'assistant':
        assistant += content

In [29]:
print(user)

do you have 5bhk in paldi?


In [30]:
print(assistant)

It appears that the document I searched does not list a 5BHK option in Paldi. The properties mentioned in the search results specify either 2BHK, 3BHK, or villas, with locations such as Bopal, Lakshmanpura, Govindpura, and Shilaj, but there is no mention of a 5BHK in Paldi. If you are specifically looking for a 5BHK in Paldi, you may want to provide a different document or more information regarding such listings.


### Delete the assistant

In [ ]:
print(client.beta.assistants.list())

In [ ]:
client.beta.assistants.delete('asst_TYkNp9sHWbyCOqboWOuLANpU')